In [1]:
import pandas as pd 
import os
import json
import sqlite3
from sqlite3 import Error
pd.set_option('display.max_rows', 200)

# Generate dataframes for insertion into db tables

In [2]:
cwd = os.getcwd()
brands_dr = cwd + '\\brands.json'
receipts_dr = cwd + '\\receipts.json'
users_dr = cwd + '\\users.json'

### Brands data

In [3]:
data = []
with open(brands_dr) as f:
    for line in f:
        data.append(json.loads(line))

In [4]:
brands_df = pd.json_normalize(data)
brands_df.head()

,barcode,category,categoryCode,name,topBrand,_id.$oid,cpg.$id.$oid,cpg.$ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


In [5]:
brands_df.columns

Index(['barcode', 'category', 'categoryCode', 'name', 'topBrand', '_id.$oid',
       'cpg.$id.$oid', 'cpg.$ref', 'brandCode'],
      dtype='object')

In [6]:
brands_cpg = brands_df[['cpg.$id.$oid','cpg.$ref']]
brands_cpg.head()

,cpg.$id.$oid,cpg.$ref
0,601ac114be37ce2ead437550,Cogs
1,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead437559,Cogs
4,5332fa12e4b03c9a25efd1e7,Cogs


In [7]:
# Final cpg_id dataframe
colrn = {'cpg.$id.$oid':'cpg_id','cpg.$ref':'cpg_ref'}
brands_cpg.rename(columns=colrn, inplace=True)
brands_cpg.head()

C:\Users\Peter\AppData\Local\Temp\ipykernel_24888\2480253218.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brands_cpg.rename(columns=colrn, inplace=True)


,cpg_id,cpg_ref
0,601ac114be37ce2ead437550,Cogs
1,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead437559,Cogs
4,5332fa12e4b03c9a25efd1e7,Cogs


In [8]:
brands_df.head()

,barcode,category,categoryCode,name,topBrand,_id.$oid,cpg.$id.$oid,cpg.$ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


In [9]:
# Final brands dataframe 
colrn = {'categoryCode':'category_code','topBrand':'top_brand','_id.$oid':'brand_id','cpg.$id.$oid':'cpg_id','cpg.$ref':'cpg_ref','brandCode':'brand_code'}
brands_df.rename(columns=colrn, inplace=True)
cols = ['brand_id','barcode','brand_code','category','category_code','cpg_id','cpg_ref','name','top_brand']
brands_df = brands_df[cols]
brands_df.head()

,brand_id,barcode,brand_code,category,category_code,cpg_id,cpg_ref,name,top_brand
0,601ac115be37ce2ead437551,511111019862,NaN,Baking,BAKING,601ac114be37ce2ead437550,Cogs,test brand @1612366101024,False
1,601c5460be37ce2ead43755f,511111519928,STARBUCKS,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Cogs,Starbucks,False
2,601ac142be37ce2ead43755d,511111819905,TEST BRANDCODE @1612366146176,Baking,BAKING,601ac142be37ce2ead437559,Cogs,test brand @1612366146176,False
3,601ac142be37ce2ead43755a,511111519874,TEST BRANDCODE @1612366146051,Baking,BAKING,601ac142be37ce2ead437559,Cogs,test brand @1612366146051,False
4,601ac142be37ce2ead43755e,511111319917,TEST BRANDCODE @1612366146827,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,Cogs,test brand @1612366146827,False


## Generate the dataframes for receipts and receipt_items table

In [10]:
receipts_df = pd.read_json(receipts_dr, lines=True)
receipts_df.head(6)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
5,{'$oid': '5ff1e1e40a7214ada1000566'},750.0,"Receipt number 1 completed, bonus point schedu...",{'$date': 1609687524000},{'$date': 1609687524000},{'$date': 1609687525000},{'$date': 1609687530000},{'$date': 1609687525000},750.0,{'$date': 1609601124000},1.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,3.25,5ff1e1e4cfcf6c399c274ac3


In [11]:
receipts_wo_items_df = receipts_df.drop(columns = "rewardsReceiptItemList")

In [12]:
receipts_wo_items_df = pd.json_normalize(receipts_wo_items_df.to_dict(orient="records"))
receipts_wo_items_df.head()

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date,finishedDate,pointsAwardedDate,purchaseDate
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,FINISHED,26.0,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,1.609632e+12,NaN,NaN,NaN
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12,NaN,NaN,NaN
2,5.0,All-receipts receipt bonus,5.0,1.0,REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,1609687537000,1609687537000,NaN,1609687542000,NaN,1.609632e+12,NaN,NaN,NaN
3,5.0,All-receipts receipt bonus,5.0,4.0,FINISHED,28.0,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,1.609632e+12,NaN,NaN,NaN
4,5.0,All-receipts receipt bonus,5.0,2.0,FINISHED,1.0,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,1.609601e+12,NaN,NaN,NaN


In [13]:
receipts_wo_items_df = receipts_wo_items_df.drop(columns = ["finishedDate","pointsAwardedDate","purchaseDate"])
receipts_wo_items_df.head()

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,FINISHED,26.0,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,1.609632e+12
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12
2,5.0,All-receipts receipt bonus,5.0,1.0,REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,1609687537000,1609687537000,NaN,1609687542000,NaN,1.609632e+12
3,5.0,All-receipts receipt bonus,5.0,4.0,FINISHED,28.0,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,1.609632e+12
4,5.0,All-receipts receipt bonus,5.0,2.0,FINISHED,1.0,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,1.609601e+12


In [14]:
receipts_wo_items_df[['createDate.$date','dateScanned.$date','finishedDate.$date','modifyDate.$date','pointsAwardedDate.$date','purchaseDate.$date']] = receipts_wo_items_df[['createDate.$date','dateScanned.$date','finishedDate.$date','modifyDate.$date','pointsAwardedDate.$date','purchaseDate.$date']].apply(pd.to_datetime, unit='ms')

receipts_wo_items_df.head()

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,FINISHED,26.0,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,2021-01-03 00:00:00
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,2021-01-02 15:24:43
2,5.0,All-receipts receipt bonus,5.0,1.0,REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,2021-01-03 00:00:00
3,5.0,All-receipts receipt bonus,5.0,4.0,FINISHED,28.0,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,2021-01-03 00:00:00
4,5.0,All-receipts receipt bonus,5.0,2.0,FINISHED,1.0,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,2021-01-02 15:25:06


In [15]:
colrn = {'bonusPointsEarned':'bonus_points_earned', 'bonusPointsEarnedReason':'bonus_points_earned_reason', 'pointsEarned':'points_earned', 'purchasedItemCount':'purchased_item_count', 'rewardsReceiptStatus':'rewards_receipt_status', 'totalSpent':'total_spent', 'userId':'user_id', '_id.$oid':'receipt_id', 'createDate.$date':'created_date', 'dateScanned.$date':'scanned_date','finishedDate.$date':'finished_date','modifyDate.$date':'modified_date','pointsAwardedDate.$date':'points_awarded_date','purchaseDate.$date':'purchase_date'}

receipts_wo_items_df.rename(columns=colrn, inplace=True)
receipts_wo_items_df.head()

,bonus_points_earned,bonus_points_earned_reason,points_earned,purchased_item_count,rewards_receipt_status,total_spent,user_id,receipt_id,created_date,scanned_date,finished_date,modified_date,points_awarded_date,purchase_date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,FINISHED,26.0,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,2021-01-03 00:00:00
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,2021-01-02 15:24:43
2,5.0,All-receipts receipt bonus,5.0,1.0,REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,2021-01-03 00:00:00
3,5.0,All-receipts receipt bonus,5.0,4.0,FINISHED,28.0,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,2021-01-03 00:00:00
4,5.0,All-receipts receipt bonus,5.0,2.0,FINISHED,1.0,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,2021-01-02 15:25:06


In [16]:
# FINAL receipts dataframe for db
cols = ['receipt_id','bonus_points_earned','bonus_points_earned_reason','created_date','finished_date','modified_date','points_awarded_date','points_earned','purchase_date','purchased_item_count','rewards_receipt_status','scanned_date','total_spent','user_id']
receipts_wo_items_df = receipts_wo_items_df[cols]
receipts_wo_items_df.head()

,receipt_id,bonus_points_earned,bonus_points_earned_reason,created_date,finished_date,modified_date,points_awarded_date,points_earned,purchase_date,purchased_item_count,rewards_receipt_status,scanned_date,total_spent,user_id
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,FINISHED,2021-01-03 15:25:31,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,FINISHED,2021-01-03 15:24:43,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,1.0,REJECTED,2021-01-03 15:25:37,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,FINISHED,2021-01-03 15:25:34,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,FINISHED,2021-01-03 15:25:06,1.0,5ff1e194b6a9d73a3a9f1052


In [17]:
receipts_wo_items_df.rewards_receipt_status.unique()

array(['FINISHED', 'REJECTED', 'FLAGGED', 'SUBMITTED', 'PENDING'],
      dtype=object)

### Receipts with items for items table

In [18]:
# generate seperate dataframe for receipts with items in order to use json_normalize
receipts_df_w_items = receipts_df[~receipts_df['rewardsReceiptItemList'].isnull()]
receipts_df_w_items.head(6)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
5,{'$oid': '5ff1e1e40a7214ada1000566'},750.0,"Receipt number 1 completed, bonus point schedu...",{'$date': 1609687524000},{'$date': 1609687524000},{'$date': 1609687525000},{'$date': 1609687530000},{'$date': 1609687525000},750.0,{'$date': 1609601124000},1.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,3.25,5ff1e1e4cfcf6c399c274ac3


In [20]:
items_norm = pd.json_normalize(receipts_df_w_items.to_dict(orient="records"), meta=['_id','purchaseDate','userId'], record_path="rewardsReceiptItemList", record_prefix="", meta_prefix="receipt_", errors='ignore')
items_norm.head(50)

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId,receipt__id,receipt_purchaseDate,receipt_userId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$oid': '5ff1e1eb0a720f0523000575'},{'$date': 1609632000000},5ff1e1eacfcf6c399c274ae6
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$oid': '5ff1e1bb0a720f052300056b'},{'$date': 1609601083000},5ff1e194b6a9d73a3a9f1052
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$oid': '5ff1e1bb0a720f052300056b'},{'$date': 1609601083000},5ff1e194b6a9d73a3a9f1052
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$oid': '5ff1e1f10a720f052300057a'},{'$date': 1609632000000},5ff1e1f1cfcf6c399c274b0b
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$oid': '5ff1e1ee0a7214ada100056f'},{'$date': 1609632000000},5ff1e1eacfcf6c399c274ae6
5,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$oid': '5ff1e1d20a7214ada1000561'},{'$date': 1609601106000},5ff1e194b6a9d73a3a9f1052
6,1234,NaN,2.56,2.56,True,2,True,3.0,1234,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$oid': '5ff1e1d20a7214ada1000561'},{'$date': 1609601106000},5ff1e194b6a9d73a3a9f1052
7,4011,ITEM NOT FOUND,3.25,3.25,False,1,True,1.0,4011,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$oid': '5ff1e1e40a7214ada1000566'},{'$date': 1609601124000},5ff1e1e4cfcf6c399c274ac3
8,NaN,MSSN TORTLLA,2.23,2.23,NaN,1009,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$oid': '5ff1e1cd0a720f052300056f'},{'$date': 1609687501000},5ff1e194b6a9d73a3a9f1052
9,046000832517,"Old El Paso Mild Chopped Green Chiles, 4.5 Oz",10.00,10.00,NaN,0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$oid': '5ff1e1a40a720f0523000569'},{'$date': 1609027200000},5ff1e194b6a9d73a3a9f1052


In [21]:
print(len(items_norm.index))

6941


In [22]:
items_norm_2 = pd.json_normalize(items_norm.to_dict(orient="records"))
items_norm_2.head()

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId,receipt_userId,receipt__id.$oid,receipt_purchaseDate.$date,receipt_purchaseDate
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,1.609632e+12,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1.609601e+12,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1.609601e+12,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,1.609632e+12,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,1.609632e+12,NaN


In [23]:
print(len(items_norm_2.index))

6941


In [24]:
items_norm_2[['receipt_purchaseDate.$date']] = items_norm_2[['receipt_purchaseDate.$date']].apply(pd.to_datetime, unit='ms')

In [25]:
colrn = {'finalPrice':'final_price', 'itemPrice':'item_price', 'needsFetchReview':'needs_fetch_review', 'partnerItemId':'partner_item_id', 'preventTargetGapPoints':'prevent_target_gap_points', 'quantityPurchased':'quantity_purchased', 'userFlaggedBarcode':'user_flagged_barcode', 'userFlaggedNewItem':'user_flagged_new_item', 'userFlaggedPrice':'user_flagged_price', 'userFlaggedQuantity':'user_flagged_quantity', 'needsFetchReviewReason':'needs_fetch_review_reason', 'pointsNotAwardedReason':'points_not_awarded_reason', 'pointsPayerId':'points_payer_id', 'rewardsGroup':'rewards_group', 'rewardsProductPartnerId':'rewards_product_partner_id', 'userFlaggedDescription':'user_flagged_description', 'originalMetaBriteBarcode':'original_metabrite_barcode', 'originalMetaBriteDescription':'original_metabrite_description', 'brandCode':'brand_code', 'competitorRewardsGroup':'competitor_rewards_group', 'discountedItemPrice':'discounted_item_price', 'originalReceiptItemText':'original_receipt_item_text', 'itemNumber':'item_number', 'originalMetaBriteQuantityPurchased':'original_metabrite_quantity_purchased', 'pointsEarned':'points_earned', 'targetPrice':'target_price', 'competitiveProduct':'competitive_product', 'originalFinalPrice':'original_final_price', 'originalMetaBriteItemPrice':'original_metabrite_item_price', 'priceAfterCoupon':'price_after_coupon', 'metabriteCampaignId':'metabrite_campaign_id', 'receipt_userId':'user_id', 'receipt__id.$oid':'receipt__id', 'receipt_purchaseDate.$date':'receipt_purchase_date'}

items_norm_2.rename(columns=colrn, inplace=True)
items_norm_2 = items_norm_2.drop(columns = "receipt_purchaseDate")
items_norm_2.head()

,barcode,description,final_price,item_price,needs_fetch_review,partner_item_id,prevent_target_gap_points,quantity_purchased,user_flagged_barcode,user_flagged_new_item,...,target_price,competitive_product,original_final_price,original_metabrite_item_price,deleted,price_after_coupon,metabrite_campaign_id,user_id,receipt__id,receipt_purchase_date
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03 00:00:00
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-02 15:24:43
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-02 15:24:43
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03 00:00:00
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03 00:00:00


In [ ]:
items_norm_2.to_csv(cwd + '\\test.tsv', sep='\t')

### Generate user table dataframe 

In [26]:
data = []
with open(users_dr) as f:
    for line in f:
        data.append(json.loads(line))
users_df = pd.json_normalize(data, errors='ignore')

users_df[['createdDate.$date','lastLogin.$date']] = users_df[['createdDate.$date','lastLogin.$date']].apply(pd.to_datetime, unit='ms')

colrn = {'_id.$oid':'user_id', 'createdDate.$date':'created_date', 'lastLogin.$date':'last_login_date', 'signUpSource':'sign_up_source'}

users_df.rename(columns=colrn, inplace=True)
users_df.head()

,active,role,sign_up_source,state,user_id,created_date,last_login_date
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.554,2021-01-03 15:25:30.596999936
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872


In [27]:
users_df.groupby(users_df.columns.tolist(),as_index=False).size()

,active,role,sign_up_source,state,user_id,created_date,last_login_date,size
0,False,consumer,Email,WI,6008622ebe5fc9247bab4eb9,2021-01-20 17:02:38.662,2021-01-20 17:02:38.931000064,1
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,11
2,True,consumer,Email,WI,5ff1e1b4cfcf6c399c274a54,2021-01-03 15:24:36.410,2021-01-03 15:24:36.452000000,1
3,True,consumer,Email,WI,5ff1e1b7cfcf6c399c274a5a,2021-01-03 15:24:39.626,2021-01-03 15:24:39.664999936,1
4,True,consumer,Email,WI,5ff1e1e4cfcf6c399c274ac3,2021-01-03 15:25:24.656,2021-01-03 15:25:24.694000128,1
5,True,consumer,Email,WI,5ff1e1e8cfcf6c399c274ad9,2021-01-03 15:25:28.354,2021-01-03 15:25:28.392000000,1
6,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.554,2021-01-03 15:25:30.596999936,4
7,True,consumer,Email,WI,5ff1e1f1cfcf6c399c274b0b,2021-01-03 15:25:37.564,2021-01-03 15:25:37.599000064,1
8,True,consumer,Email,WI,5ff36a3862fde912123a4460,2021-01-04 19:19:20.352,2021-01-04 19:23:05.340000000,5
9,True,consumer,Email,WI,5ff36be7135e7011bcb856d3,2021-01-04 19:26:31.239,2021-01-04 19:29:52.726000128,3


In [28]:
users_df = users_df.drop_duplicates()
users_df.count()

active             212
role               212
sign_up_source     207
state              206
user_id            212
created_date       212
last_login_date    172
dtype: int64

In [29]:
cols = ['user_id','active','created_date','last_login_date','role','sign_up_source','state']
users_df = users_df[cols]

In [30]:
users_df.head()

,user_id,active,created_date,last_login_date,role,sign_up_source,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554,2021-01-03 15:25:30.596999936,consumer,Email,WI
6,5ff1e1e8cfcf6c399c274ad9,True,2021-01-03 15:25:28.354,2021-01-03 15:25:28.392000000,consumer,Email,WI
7,5ff1e1b7cfcf6c399c274a5a,True,2021-01-03 15:24:39.626,2021-01-03 15:24:39.664999936,consumer,Email,WI
9,5ff1e1f1cfcf6c399c274b0b,True,2021-01-03 15:25:37.564,2021-01-03 15:25:37.599000064,consumer,Email,WI


## Create Sqlite db

In [31]:
fp = cwd + '\\fetch_sqlite.db'
""" create a database connection to a SQLite database """
conn = None
try:
    conn = sqlite3.connect(fp)
    print(sqlite3.version)
except Error as e:
    print(e)
finally:
    if conn:
        conn.close()

2.6.0


## Generate Tables

In [32]:
users_df.head()

,user_id,active,created_date,last_login_date,role,sign_up_source,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554,2021-01-03 15:25:30.596999936,consumer,Email,WI
6,5ff1e1e8cfcf6c399c274ad9,True,2021-01-03 15:25:28.354,2021-01-03 15:25:28.392000000,consumer,Email,WI
7,5ff1e1b7cfcf6c399c274a5a,True,2021-01-03 15:24:39.626,2021-01-03 15:24:39.664999936,consumer,Email,WI
9,5ff1e1f1cfcf6c399c274b0b,True,2021-01-03 15:25:37.564,2021-01-03 15:25:37.599000064,consumer,Email,WI


In [33]:
brands_df.head()

,brand_id,barcode,brand_code,category,category_code,cpg_id,cpg_ref,name,top_brand
0,601ac115be37ce2ead437551,511111019862,NaN,Baking,BAKING,601ac114be37ce2ead437550,Cogs,test brand @1612366101024,False
1,601c5460be37ce2ead43755f,511111519928,STARBUCKS,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Cogs,Starbucks,False
2,601ac142be37ce2ead43755d,511111819905,TEST BRANDCODE @1612366146176,Baking,BAKING,601ac142be37ce2ead437559,Cogs,test brand @1612366146176,False
3,601ac142be37ce2ead43755a,511111519874,TEST BRANDCODE @1612366146051,Baking,BAKING,601ac142be37ce2ead437559,Cogs,test brand @1612366146051,False
4,601ac142be37ce2ead43755e,511111319917,TEST BRANDCODE @1612366146827,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,Cogs,test brand @1612366146827,False


In [34]:
brands_cpg.head()

,cpg_id,cpg_ref
0,601ac114be37ce2ead437550,Cogs
1,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead437559,Cogs
4,5332fa12e4b03c9a25efd1e7,Cogs


In [35]:
receipts_wo_items_df.head()

,receipt_id,bonus_points_earned,bonus_points_earned_reason,created_date,finished_date,modified_date,points_awarded_date,points_earned,purchase_date,purchased_item_count,rewards_receipt_status,scanned_date,total_spent,user_id
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,FINISHED,2021-01-03 15:25:31,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,FINISHED,2021-01-03 15:24:43,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,1.0,REJECTED,2021-01-03 15:25:37,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,FINISHED,2021-01-03 15:25:34,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,FINISHED,2021-01-03 15:25:06,1.0,5ff1e194b6a9d73a3a9f1052


In [ ]:
c = conn.cursor()
try:
    c.executescript('''
              CREATE TABLE IF NOT EXISTS users (
              user_id TEXT PRIMARY KEY,
              active INTEGER,
              created_date TEXT,
              last_login_date TEXT,
              role TEXT,
              sign_up_source TEXT,
              state TEXT
              );
              
              CREATE TABLE IF NOT EXISTS brands (
              brand_id TEXT PRIMARY KEY,
              barcode TEXT,
              brand_code TEXT,
              category TEXT,
              category_code TEXT,
              cpg_id TEXT,
              cpg_ref TEXT,
              name TEXT,
              top_brand TEXT
              );
              
              CREATE TABLE IF NOT EXISTS brand_cpg (
              cpg_id TEXT,
              cpg_ref TEXT
              );
              
              CREATE TABLE IF NOT EXISTS receipts (
              receipt_id TEXT PRIMARY KEY,
              bonus_points_earned REAL,
              bonus_points_earned_reason TEXT,
              created_date TEXT,
              finished_date TEXT,
              modified_date TEXT,
              points_awarded_date TEXT,
              points_earned REAL,
              purchase_date TEXT,
              purchased_item_count REAL,
              rewards_receipt_status TEXT,
              scanned_date TEXT,
              total_spent REAL,
              user_id TEXT
              );
              
              CREATE TABLE IF NOT EXISTS receipt_items (
              barcode INTEGER,
              description TEXT,
              final_price REAL,
              item_price REAL,
              needs_fetch_review INTEGER,
              partner_item_id TEXT,
              prevent_target_gap_points INTEGER,
              quantity_purchased INTEGER,
              user_flagged_barcode INTEGER,
              user_flagged_new_item INTEGER,
              user_flagged_price REAL,
              user_flagged_quantity INTEGER,
              needs_fetch_review_reason TEXT,
              points_not_awarded_reason TEXT,
              points_payer_id TEXT,
              rewards_group TEXT,
              rewards_product_partner_id TEXT,
              user_flagged_description TEXT,
              original_metabrite_barcode INTEGER,
              original_metabrite_description TEXT,
              brand_code TEXT,
              competitor_rewards_group TEXT,
              discounted_item_price REAL,
              original_receipt_item_text TEXT,
              item_number INTEGER,
              original_metabrite_quantity_purchased INTEGER,
              points_earned REAL,
              target_price REAL,
              competitive_product INTEGER,
              original_final_price REAL,
              original_metabrite_item_price REAL,
              deleted INTEGER,
              price_after_coupon REAL,
              metabrite_campaign_id TEXT,
              user_id TEXT,
              receipt_id TEXT,
              receipt_purchase_date TEXT
              );
              ''')
except Error as e:
    print(e)

## Load Data Into Tables

In [ ]:
users_df[['created_date', 'last_login_date']] = users_df[['created_date', 'last_login_date']].astype(str)

In [ ]:
users_df.to_sql('users', if_exists='append', con=conn, index=False)

In [ ]:
receipts_wo_items_df.to_sql('receipts', if_exists='append', con=conn, index=False)

In [ ]:
items_norm_2.to_sql('receipt_items', if_exists='append', con=conn, index=False)

In [ ]:
brands_cpg.to_sql('brand_cpg', if_exists='append', con=conn, index=False)

In [ ]:
brands_df.to_sql('brands', if_exists='append', con=conn, index=False)

In [36]:
items_norm_2[items_norm_2.duplicated()]

,barcode,description,final_price,item_price,needs_fetch_review,partner_item_id,prevent_target_gap_points,quantity_purchased,user_flagged_barcode,user_flagged_new_item,...,target_price,competitive_product,original_final_price,original_metabrite_item_price,deleted,price_after_coupon,metabrite_campaign_id,user_id,receipt__id,receipt_purchase_date
